In [0]:
from google.colab import drive 
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
import numpy as np
import pandas as pd
%cd "/content/gdrive/My Drive/thesis/Data"

#csv import labels

train_ACL_labels = np.array(pd.read_csv("train-acl.csv", header=None).iloc[:,1])
train_abnormal_labels = np.array(pd.read_csv("train-abnormal.csv", header=None).iloc[:,1])
train_meniscus_labels = np.array(pd.read_csv("train-meniscus.csv", header=None).iloc[:,1])

valid_ACL_labels = np.array(pd.read_csv("valid-acl.csv", header=None).iloc[:,1])
valid_abnormal_labels = np.array(pd.read_csv("valid-abnormal.csv", header=None).iloc[:,1])
valid_meniscus_labels = np.array(pd.read_csv("valid-meniscus.csv", header=None).iloc[:,1])

test_ACL_labels = np.array(pd.read_csv("test-acl.csv", header=None).iloc[:,1])
test_abnormal_labels = np.array(pd.read_csv("test-abnormal.csv", header=None).iloc[:,1])
test_meniscus_labels = np.array(pd.read_csv("test-meniscus.csv", header=None).iloc[:,1])

#data path
train_path = "/content/gdrive/My Drive/thesis/Data/train"
train_axial_path = "/content/gdrive/My Drive/thesis/Data/train/coronal"

counter = 5
for filename in os.listdir(train_axial_path):
  if counter > 0:
    file0 = np.load(train_axial_path + '/' + filename)
    variancelist = []
    for slice in range(file0.shape[0]):
      variancelist.append(np.var(file0[slice,:,:]))
    #print(file0)
    #print('max', np.amax(file0))
    #print('mean', np.mean(file0))
    #print(filename, 'file shape', file0.shape, 'max_var_index', variancelist.index(max(variancelist)), max(variancelist))
    counter = counter - 1

print(valid_ACL_labels)
print("done!")

/content/gdrive/My Drive/thesis/Data
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 1 1 0 1 0]
done!


In [0]:
# model.py
import numpy as np
import torch
import torch.nn as nn

from torchvision import models

torch.set_printoptions(precision=10)

class MRNet(nn.Module):
    def __init__(self, custommodel='alexnet'):
        super().__init__()
        if custommodel == 'alexnet':
          self.model = models.alexnet(pretrained=True)
          self.classifier = nn.Linear(256,1)
        if custommodel == 'vgg11_bn':
          self.model = models.vgg11_bn(pretrained=True)
          self.classifier = nn.Linear(512,1)
        self.gap = nn.AdaptiveAvgPool2d(1)

    # change this to adapt to different networks
    def forward(self, x, savedir='None'):
        x = torch.squeeze(x, dim=0) # only batch size 1 supported
        x = self.model.features(x)
        # make sure that gap returns size 256
        x = self.gap(x).view(x.size(0), -1)
        x = torch.max(x, 0, keepdim=True)[0]
        #features3 = (x.cpu()).detach().numpy()
        x = self.classifier(x)
        #print('x name', x)
        #print('x torch sigmoid', torch.sigmoid(x))
        #if savedir != 'None':
        #  np.save(savedir + str(x), features3)
        return x

In [0]:
import math
import torch
from torch.optim.optimizer import Optimizer, required

class RAdam(Optimizer):

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0, degenerated_to_sgd=True):
        if not 0.0 <= lr:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if not 0.0 <= eps:
            raise ValueError("Invalid epsilon value: {}".format(eps))
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError("Invalid beta parameter at index 0: {}".format(betas[0]))
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError("Invalid beta parameter at index 1: {}".format(betas[1]))
        
        self.degenerated_to_sgd = degenerated_to_sgd
        if isinstance(params, (list, tuple)) and len(params) > 0 and isinstance(params[0], dict):
            for param in params:
                if 'betas' in param and (param['betas'][0] != betas[0] or param['betas'][1] != betas[1]):
                    param['buffer'] = [[None, None, None] for _ in range(10)]
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay, buffer=[[None, None, None] for _ in range(10)])
        super(RAdam, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(RAdam, self).__setstate__(state)

    def step(self, closure=None):

        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:

            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data.float()
                if grad.is_sparse:
                    raise RuntimeError('RAdam does not support sparse gradients')

                p_data_fp32 = p.data.float()

                state = self.state[p]

                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p_data_fp32)
                    state['exp_avg_sq'] = torch.zeros_like(p_data_fp32)
                else:
                    state['exp_avg'] = state['exp_avg'].type_as(p_data_fp32)
                    state['exp_avg_sq'] = state['exp_avg_sq'].type_as(p_data_fp32)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
                exp_avg.mul_(beta1).add_(1 - beta1, grad)

                state['step'] += 1
                buffered = group['buffer'][int(state['step'] % 10)]
                if state['step'] == buffered[0]:
                    N_sma, step_size = buffered[1], buffered[2]
                else:
                    buffered[0] = state['step']
                    beta2_t = beta2 ** state['step']
                    N_sma_max = 2 / (1 - beta2) - 1
                    N_sma = N_sma_max - 2 * state['step'] * beta2_t / (1 - beta2_t)
                    buffered[1] = N_sma

                    # more conservative since it's an approximated value
                    if N_sma >= 5:
                        step_size = math.sqrt((1 - beta2_t) * (N_sma - 4) / (N_sma_max - 4) * (N_sma - 2) / N_sma * N_sma_max / (N_sma_max - 2)) / (1 - beta1 ** state['step'])
                    elif self.degenerated_to_sgd:
                        step_size = 1.0 / (1 - beta1 ** state['step'])
                    else:
                        step_size = -1
                    buffered[2] = step_size

                # more conservative since it's an approximated value
                if N_sma >= 5:
                    if group['weight_decay'] != 0:
                        p_data_fp32.add_(-group['weight_decay'] * group['lr'], p_data_fp32)
                    denom = exp_avg_sq.sqrt().add_(group['eps'])
                    p_data_fp32.addcdiv_(-step_size * group['lr'], exp_avg, denom)
                    p.data.copy_(p_data_fp32)
                elif step_size > 0:
                    if group['weight_decay'] != 0:
                        p_data_fp32.add_(-group['weight_decay'] * group['lr'], p_data_fp32)
                    p_data_fp32.add_(-step_size * group['lr'], exp_avg)
                    p.data.copy_(p_data_fp32)

        return loss

class PlainRAdam(Optimizer):

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0, degenerated_to_sgd=True):
        if not 0.0 <= lr:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if not 0.0 <= eps:
            raise ValueError("Invalid epsilon value: {}".format(eps))
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError("Invalid beta parameter at index 0: {}".format(betas[0]))
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError("Invalid beta parameter at index 1: {}".format(betas[1]))
                    
        self.degenerated_to_sgd = degenerated_to_sgd
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay)

        super(PlainRAdam, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(PlainRAdam, self).__setstate__(state)

    def step(self, closure=None):

        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:

            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data.float()
                if grad.is_sparse:
                    raise RuntimeError('RAdam does not support sparse gradients')

                p_data_fp32 = p.data.float()

                state = self.state[p]

                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p_data_fp32)
                    state['exp_avg_sq'] = torch.zeros_like(p_data_fp32)
                else:
                    state['exp_avg'] = state['exp_avg'].type_as(p_data_fp32)
                    state['exp_avg_sq'] = state['exp_avg_sq'].type_as(p_data_fp32)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
                exp_avg.mul_(beta1).add_(1 - beta1, grad)

                state['step'] += 1
                beta2_t = beta2 ** state['step']
                N_sma_max = 2 / (1 - beta2) - 1
                N_sma = N_sma_max - 2 * state['step'] * beta2_t / (1 - beta2_t)


                # more conservative since it's an approximated value
                if N_sma >= 5:
                    if group['weight_decay'] != 0:
                        p_data_fp32.add_(-group['weight_decay'] * group['lr'], p_data_fp32)
                    step_size = group['lr'] * math.sqrt((1 - beta2_t) * (N_sma - 4) / (N_sma_max - 4) * (N_sma - 2) / N_sma * N_sma_max / (N_sma_max - 2)) / (1 - beta1 ** state['step'])
                    denom = exp_avg_sq.sqrt().add_(group['eps'])
                    p_data_fp32.addcdiv_(-step_size, exp_avg, denom)
                    p.data.copy_(p_data_fp32)
                elif self.degenerated_to_sgd:
                    if group['weight_decay'] != 0:
                        p_data_fp32.add_(-group['weight_decay'] * group['lr'], p_data_fp32)
                    step_size = group['lr'] / (1 - beta1 ** state['step'])
                    p_data_fp32.add_(-step_size, exp_avg)
                    p.data.copy_(p_data_fp32)

        return loss


class AdamW(Optimizer):

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0, warmup = 0):
        if not 0.0 <= lr:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if not 0.0 <= eps:
            raise ValueError("Invalid epsilon value: {}".format(eps))
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError("Invalid beta parameter at index 0: {}".format(betas[0]))
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError("Invalid beta parameter at index 1: {}".format(betas[1]))
        
        defaults = dict(lr=lr, betas=betas, eps=eps,
                        weight_decay=weight_decay, warmup = warmup)
        super(AdamW, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(AdamW, self).__setstate__(state)

    def step(self, closure=None):
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:

            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data.float()
                if grad.is_sparse:
                    raise RuntimeError('Adam does not support sparse gradients, please consider SparseAdam instead')

                p_data_fp32 = p.data.float()

                state = self.state[p]

                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p_data_fp32)
                    state['exp_avg_sq'] = torch.zeros_like(p_data_fp32)
                else:
                    state['exp_avg'] = state['exp_avg'].type_as(p_data_fp32)
                    state['exp_avg_sq'] = state['exp_avg_sq'].type_as(p_data_fp32)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                state['step'] += 1

                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
                exp_avg.mul_(beta1).add_(1 - beta1, grad)

                denom = exp_avg_sq.sqrt().add_(group['eps'])
                bias_correction1 = 1 - beta1 ** state['step']
                bias_correction2 = 1 - beta2 ** state['step']
                
                if group['warmup'] > state['step']:
                    scheduled_lr = 1e-8 + state['step'] * group['lr'] / group['warmup']
                else:
                    scheduled_lr = group['lr']

                step_size = scheduled_lr * math.sqrt(bias_correction2) / bias_correction1
                
                if group['weight_decay'] != 0:
                    p_data_fp32.add_(-group['weight_decay'] * scheduled_lr, p_data_fp32)

                p_data_fp32.addcdiv_(-step_size, exp_avg, denom)

                p.data.copy_(p_data_fp32)

        return loss

In [0]:
import argparse
import json
import numpy as np
import os
import torch
from datetime import datetime
from pathlib import Path
from sklearn import metrics

def train(rundir, diagnosis, orientation, epochs, learning_rate, transformbool, use_gpu):
    
    val_auc_array = list()
    train_auc_array = list()
    test_auc_array = list()

    train_fpr_array = list()
    train_tpr_array = list()
    val_fpr_array = list()
    val_tpr_array = list()
    test_fpr_array = list()
    test_tpr_array = list()

    train_loader, valid_loader, test_loader = load_data(diagnosis, orientation, transformbool, use_gpu)
    
    model = MRNet()

    if use_gpu:
        model = model.cuda()

    optimizer = RAdam(model.parameters(), learning_rate, weight_decay=.01)

    # patience too low (after 5 epochs, if AUC hasnt improved, slash learning rate .3), which is why high learning rate seems to work better
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=10, factor=.3, threshold=1e-4)

    best_val_loss = float('inf')

    start_time = datetime.now()

    for epoch in range(epochs):
        change = datetime.now() - start_time
        print('starting epoch {}. time passed: {}'.format(epoch+1, str(change)))
        
        train_loss, train_auc, _, _, tra_fpr, tra_tpr = run_model(model, train_loader, train=True, optimizer=optimizer)
        #print(f'train loss: {train_loss:0.4f}')
        #print(f'train AUC: {train_auc:0.4f}')

        val_loss, val_auc, _, _, val_fpr, val_tpr = run_model(model, valid_loader)
        #print(f'valid loss: {val_loss:0.4f}')
        #print(f'valid AUC: {val_auc:0.4f}')

        test_loss, test_auc, _, _, te_fpr, te_tpr = run_model(model, test_loader)

        val_auc_array.append(val_auc)
        train_auc_array.append(train_auc)
        test_auc_array.append(test_auc)
        train_fpr_array.append(tra_fpr)
        train_tpr_array.append(tra_tpr)
        val_fpr_array.append(val_fpr)
        val_tpr_array.append(val_tpr)
        test_fpr_array.append(te_fpr)
        test_tpr_array.append(te_tpr)
        
        scheduler.step(val_loss)
  
    
    file_name = f'val{val_auc:0.4f}_train{train_auc:0.4f}_test{test_auc:0.4f}_epoch{epoch+1}'
    save_path = "/content/gdrive/My Drive/thesis/Results/models" + '/' + str(diagnosis) + '/' + str(orientation) + "/" + file_name
    torch.save(model.state_dict(), save_path)
    print('model saved at', str(save_path))
        
    return val_auc_array, train_auc_array, test_auc_array, train_fpr_array, train_tpr_array, val_fpr_array, val_tpr_array, test_fpr_array, test_tpr_array

In [0]:
# loader.py

!pip install medicaltorch

import numpy as np
import os
import pickle
import torch
import torch.nn.functional as F
import torch.utils.data as data
import torchvision
from medicaltorch import transforms as mt_transforms
import PIL
from random import sample

from torch.autograd import Variable

INPUT_DIM = 224
MAX_PIXEL_VAL = 255
#MEAN = 58.09
#STDDEV = 49.73

class Dataset(data.Dataset):
    def __init__(self, datadirs, diagnosis, orientation, use_gpu, transformbool):
        super().__init__()
        self.use_gpu = use_gpu
        self.transformbool = transformbool
        label_dict = {}
        self.paths = []
        #print(datadirs)
        
        self.orientation = orientation
        self.diagnosis = diagnosis

        train_string = "/content/gdrive/My Drive/thesis/Data/train"
        valid_string = "/content/gdrive/My Drive/thesis/Data/valid"
        test_string = "/content/gdrive/My Drive/thesis/Data/test"

        if datadirs == train_string:
          if diagnosis == 'ACL':
            self.labels = train_ACL_labels
          if diagnosis == 'meniscus':
            self.labels = train_meniscus_labels
          if diagnosis == 'abnormal':
            self.labels = train_abnormal_labels
        if datadirs == valid_string:
          if diagnosis == 'ACL':
            self.labels = valid_ACL_labels
          if diagnosis == 'meniscus':
            self.labels = valid_meniscus_labels
          if diagnosis == 'abnormal':
            self.labels = valid_abnormal_labels
        if datadirs == test_string:
          if diagnosis == 'ACL':
            self.labels = test_ACL_labels
          if diagnosis == 'meniscus':
            self.labels = test_meniscus_labels
          if diagnosis == 'abnormal':
            self.labels = test_abnormal_labels

        direct = datadirs + '/' + self.orientation
        for file in os.listdir(direct):
          self.paths.append(direct + '/' + file)
        self.paths.sort()

        #print("paths", self.paths[0:10])

        neg_weight = np.mean(self.labels)
        self.weights = [neg_weight, 1 - neg_weight]

        #print(self.labels.shape)
        #print(self.weights)

    def weighted_loss(self, prediction, target):
        weights_npy = np.array([self.weights[int(t[0])] for t in target.data])
        weights_tensor = torch.FloatTensor(weights_npy)
        if self.use_gpu:
            weights_tensor = weights_tensor.cuda()
        loss = F.binary_cross_entropy_with_logits(prediction, target, weight=Variable(weights_tensor))
        return loss

    # Data augmentation section
    # can go through each cases, looking at the histogram of 3T vs 1.5T (naive distribution of contrast data?)
    def __getitem__(self, index):
        path = self.paths[index]
        vol = np.load(path)

        ax_mean = 63.16
        ax_std = 60.46
        cor_mean = 59.27
        cor_std = 64.00
        sag_mean = 58.25
        sag_std = 48.15

        # standardize
        vol = (vol - np.min(vol)) / (np.max(vol) - np.min(vol) + 1.0e-6) * MAX_PIXEL_VAL
        
        if self.orientation == 'axial':
          MEAN = ax_mean
          STDDEV = ax_std
        if self.orientation == 'coronal':
          MEAN = cor_mean
          STDDEV = cor_std
        if self.orientation == 'sagittal':
          MEAN = sag_mean
          STDDEV = sag_std
        
        vol = (vol - MEAN) / STDDEV

        vol = vol.astype(np.float32)

        flag = False
        randomangle = 0

        # define transform policy
        hor_flip = np.random.rand(1)
        ran_rot = np.random.rand(1)
        randomangle = np.random.uniform(-20, 20)
        uni_noise = np.random.rand(1)

        """
        if ran_rot < 0.5:
          randomangle = 0
        """
        if self.transformbool:
          #if np.random.rand(1) < 0.5:
          flag = True
          
          if uni_noise < 0.5:
            noise_array = np.random.uniform(0.95,1.05,256*256)
            noise_array.resize((256,256))
            
            vol = np.multiply(vol, noise_array)
            vol = np.clip(vol, 0, 255)
            vol = vol.astype(np.float32)
          
          self.transforms = torchvision.transforms.Compose([
            torchvision.transforms.ToPILImage(),
            torchvision.transforms.RandomHorizontalFlip(p=(hor_flip < 0.5)), 
            torchvision.transforms.RandomAffine((randomangle,randomangle), resample=PIL.Image.BILINEAR),
            torchvision.transforms.ToTensor()
        ])

        if flag:
          for sliceindex in range(vol.shape[0]):
            vol[sliceindex] = self.transforms(np.array(vol[sliceindex]))

        vol = np.stack((vol,)*3, axis=1)
        vol_tensor = torch.FloatTensor(vol)
        label_tensor = torch.FloatTensor([self.labels[index]])

        return vol_tensor, label_tensor

    def __len__(self):
        return len(self.paths)

def load_data(diagnosis, orientation, transformbool, use_gpu=True):

    print('load_data', diagnosis, orientation)

    train_path = "/content/gdrive/My Drive/thesis/Data/train"
    valid_path = "/content/gdrive/My Drive/thesis/Data/valid"
    test_path = "/content/gdrive/My Drive/thesis/Data/test"

    batchsize = 1
    numworkers = 8
    
    #assert(1==2)
    #train_dataset = Dataset(train_dirs, diagnosis, use_gpu)
    train_dataset = Dataset(train_path, diagnosis, orientation, use_gpu, transformbool)
    valid_dataset = Dataset(valid_path, diagnosis, orientation, use_gpu, False)
    test_dataset = Dataset(test_path, diagnosis, orientation, use_gpu, False)

    train_loader = data.DataLoader(train_dataset, batch_size=batchsize, num_workers=numworkers, shuffle=False)
    valid_loader = data.DataLoader(valid_dataset, batch_size=batchsize, num_workers=numworkers, shuffle=False)
    test_loader = data.DataLoader(test_dataset, batch_size=batchsize, num_workers=numworkers, shuffle=False)
    return train_loader, valid_loader, test_loader


In [0]:
# evaluate.py

import argparse
import matplotlib.pyplot as plt
import os
import numpy as np
import torch

from sklearn import metrics
from torch.autograd import Variable

#from loader import load_data
#from model import MRNet

def get_parser():
    parser = argparse.ArgumentParser()
    parser.add_argument('--model_path', type=str, required=True)
    parser.add_argument('--split', type=str, required=True)
    parser.add_argument('--diagnosis', type=int, required=True)
    parser.add_argument('--gpu', action='store_true')
    return parser

def run_model(model, loader, savedir='None', train=False, optimizer=None):
    preds = []
    labels = []

    if train:
        model.train()
    else:
        model.eval()

    total_loss = 0.
    num_batches = 0

    for batch in loader:
        if train:
            optimizer.zero_grad()

        vol, label = batch
        if loader.dataset.use_gpu:
            vol = vol.cuda()
            label = label.cuda()
        vol = Variable(vol)
        label = Variable(label)

        logit = model.forward(vol, savedir)

        loss = loader.dataset.weighted_loss(logit, label)
        total_loss += loss.item()

        #
        pred = torch.sigmoid(logit)
        pred_npy = pred.data.cpu().numpy()[0][0]
        label_npy = label.data.cpu().numpy()[0][0]

        preds.append(pred_npy)
        labels.append(label_npy)

        if train:
            loss.backward()
            optimizer.step()
        num_batches += 1

    avg_loss = total_loss / num_batches

    fpr, tpr, threshold = metrics.roc_curve(labels, preds)
    print("threshold is: ", threshold)
    auc = metrics.auc(fpr, tpr)

    return avg_loss, auc, preds, labels, fpr, tpr

def evaluate(split, model_path, diagnosis, orientation, transformbool, use_gpu, savedir):

    train_loader, valid_loader, test_loader = load_data(diagnosis, orientation, transformbool, use_gpu)
    try:
      model = MRNet(custommodel='alexnet')
      print('using alexnet ...')
    except:
      model = MRNet(custommodel='vgg11_bn')
      print('using vgg11_bn')
    state_dict = torch.load(model_path, map_location=(None if use_gpu else 'cpu'))
    model.load_state_dict(state_dict)

    if use_gpu:
        model = model.cuda()

    if split == 'train':
        loader = train_loader
    elif split == 'valid':
        loader = valid_loader
    elif split == 'test':
        loader = test_loader
    else:
        raise ValueError("split must be 'train', 'valid', or 'test'")

    loss, auc, preds, labels, fpr, tpr = run_model(model, loader, savedir)

    print(f'{split} loss: {loss:0.4f}')
    print(f'{split} AUC: {auc:0.4f}')

    print("FPR: ", fpr)
    print("TPR: ", tpr)

    return preds, labels

#if __name__ == '__main__':
#    args = get_parser().parse_args()
#   evaluate(args.split, args.model_path, args.diagnosis, args.gpu)

In [0]:
import matplotlib
matplotlib.use('Agg')
gpu = True
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

#learningrate = 5e-05
epochs = 100
#diagnosis = 'ACL'
rundir = "/content/gdrive/My Drive/thesis/Data"
#orientation = 'axial'
savedir = "/content/gdrive/My Drive/thesis/Results/round2"


if gpu:
  torch.cuda.manual_seed_all(seed)

def display_single(x_length, lr1, varray, tarray, testarray, title, xlabel, ylabel, save_dir):
  plt.figure(0)
  plt.title(title)
  plt.ylim(0.0,1.0)
  plt.plot(np.arange(x_length), varray, label='valid')
  plt.plot(np.arange(x_length), tarray, label='train')
  plt.plot(np.arange(x_length), testarray, label='test')
  plt.legend()
  plt.xlabel(xlabel)
  plt.ylabel(ylabel)
  plt.savefig(save_dir + '/' + title + '.eps', format='eps')
  plt.show()
  plt.close()
  return

Change Transforms


In [0]:
import matplotlib
matplotlib.use('Agg')
gpu = True
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

#learningrate = 5e-05
epochs = 100
#diagnosis = 'ACL'
rundir = "/content/gdrive/My Drive/thesis/Data"
#orientation = 'axial'
savedir = "/content/gdrive/My Drive/thesis/Results/round3"


if gpu:
  torch.cuda.manual_seed_all(seed)

def display_single(x_length, lr1, varray, tarray, testarray, title, xlabel, ylabel, save_dir):
  plt.figure(0)
  plt.title(title)
  plt.ylim(0.0,1.0)
  plt.plot(np.arange(x_length), varray, label='valid')
  plt.plot(np.arange(x_length), tarray, label='train')
  plt.plot(np.arange(x_length), testarray, label='test')
  plt.legend()
  plt.xlabel(xlabel)
  plt.ylabel(ylabel)
  plt.savefig(save_dir + '/' + title + '.eps', format='eps')
  plt.show()
  plt.close()
  return

In [0]:
# find models for 1 disease, with best performing validation set performance
# evaluate these models on the data to produce prediction labels for all three orientations
# use set of 3 label lists to predict on ultimate label
## mean, mode, logistic regression, neural net, 

In [0]:
models_path = "/content/gdrive/My Drive/thesis/Results/models"
diags = ['ACL', 'meniscus', 'abnormal']
oris = ['axial', 'sagittal', 'coronal']

def choose_models(diag):
  # ax_path, sag_path, cor_path
  return_list = list()
  for ori in oris:
    temp_path = models_path + '/' + diag + '/' + ori
    rank_list = list()
    path_list = list()

    for filename in os.listdir(temp_path):
      val, train, test, epoch = [entry.split('.') for entry in filename.split('_')]
      # if want to sort by highest validation AUC
      # could probably change it to something like validation * 5 - train * 3 + test or something along this line
      #rank_list.append(5* int(val[1]) - 3* (int(train[1]) - int(test[1])))
      rank_list.append(int(val[1]) - 0.2*abs(int(train[1]) - int(val[1])))
      path_list.append(temp_path + '/' + filename)
    
    maxi = rank_list.index(max(rank_list))
    #print(diag, ori, maxi, rank_list[maxi], path_list[maxi])
    return_list.append(path_list[maxi])

  return return_list

In [0]:
def evaluate_three(diagnosis):
  modelpathslist = choose_models(diagnosis)
  oris = ['axial', 'sagittal', 'coronal']
  splits = ['train', 'valid', 'test']
  predslist = list()
  labelslist = list()
  start_time = datetime.now()

  for split in splits:
    for index in range(3):
      print(split, oris[index])
      change = datetime.now() - start_time
      #print('time passed: {}'.format(str(change)))
      savedir = "/content/gdrive/My Drive/thesis/Results/model_features/" + str(diagnosis) + '/' + str(split) + '/' + str(oris[index]) + '/'
      preds, labels = evaluate(split, modelpathslist[index], diagnosis, oris[index], False, True, savedir)
      save_preds = '/content/gdrive/My Drive/thesis/Results/model_predictions/' + str(diagnosis) + '/' + str(split) + '/' + str(oris[index]) + '/' + 'predictions.npy'
      np.save(save_preds, preds)
      predslist.append([preds])
      labelslist.append([labels])

  return predslist, labelslist

In [0]:
predslist_ACL, labelslist_ACL = evaluate_three('ACL')

train axial
load_data ACL axial


Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /root/.cache/torch/checkpoints/alexnet-owt-4df8aa71.pth



using alexnet ...
threshold is:  [1.9999987e+00 9.9999869e-01 9.9989629e-01 9.9989533e-01 9.9571663e-01
 9.9552476e-01 9.9419159e-01 9.9413562e-01 9.9398613e-01 9.9369109e-01
 9.8694009e-01 9.8682320e-01 9.8629123e-01 9.8619080e-01 9.8605376e-01
 9.8477978e-01 9.8404467e-01 9.8292357e-01 9.7803444e-01 9.7799623e-01
 9.7451383e-01 9.7351968e-01 9.7324592e-01 9.7038561e-01 9.7024035e-01
 9.6864843e-01 9.6780425e-01 9.6676773e-01 9.6484852e-01 9.6276546e-01
 9.6195531e-01 9.6052414e-01 9.5970494e-01 9.5948130e-01 9.5931268e-01
 9.5909911e-01 9.5408475e-01 9.5401251e-01 9.5341158e-01 9.5002979e-01
 9.4885296e-01 9.4697195e-01 9.4363028e-01 9.3843168e-01 9.3481541e-01
 9.3157858e-01 9.3038768e-01 9.3016607e-01 9.2654383e-01 9.2167944e-01
 9.2134649e-01 9.1360879e-01 9.1334587e-01 9.1178173e-01 9.1084802e-01
 9.0816790e-01 9.0742785e-01 9.0720999e-01 9.0719903e-01 9.0139520e-01
 9.0067708e-01 8.9646900e-01 8.9535922e-01 8.9117783e-01 8.9068246e-01
 8.8628501e-01 8.8385528e-01 8.8200533e-01 

In [0]:
print('ACL prediction values: train 3 oris, valid 3 oris, test 3 oris')
print(np.array(predslist_ACL))
print(np.array(predslist_ACL).shape)

ACL prediction values: train 3 oris, valid 3 oris, test 3 oris
[[list([0.001057625, 0.9956304, 0.0014649852, 0.10609255, 0.006067098, 0.0033691446, 0.009125966, 0.00021953894, 0.00025437216, 0.0054137125, 0.017695615, 0.05560858, 0.04184516, 0.084910415, 0.0036287094, 0.070385285, 0.013959859, 5.4142356e-06, 0.95615065, 0.054029137, 0.06766702, 0.0009817217, 0.0073269075, 7.367659e-05, 0.0019456818, 0.00010642096, 0.0024543295, 0.018031355, 0.01651568, 0.3127834, 0.0019737387, 0.00036177604, 0.006587421, 0.000115697374, 0.035576295, 0.005240455, 0.022196058, 0.9912947, 7.245637e-05, 0.0050794347, 0.003932356, 0.00016025078, 0.2022345, 0.27289587, 0.00059744925, 0.0021018132, 0.9958682, 0.004167748, 0.003395042, 0.9971834, 0.0034349766, 0.0093552675, 0.0022203363, 0.00032495713, 0.09394123, 0.00019477858, 0.0011986769, 0.004269341, 1.2618831e-05, 0.9999347, 9.756829e-05, 0.00208196, 0.993174, 0.0037781529, 0.00021775464, 0.0014450606, 0.00095682114, 0.011020652, 0.08001484, 1.4154802e-0

In [0]:
print('ACL labels: train 3 oris, valid 3 oris, test 3 oris')
print(np.array(labelslist_ACL))
print(np.array(labelslist_ACL).shape)

ACL labels: train 3 oris, valid 3 oris, test 3 oris
[[list([0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 

In [0]:
for index in range(9):
  # find indices where prediction value was different from label
  abslist = abs(np.subtract(np.array(predslist_ACL[index]),np.array(labelslist_ACL[index]))) 
  returnlist = np.where(abslist < 0.5, 0, 1)
  print(np.sum(returnlist), returnlist)

1 [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [0]:
predslist_meniscus, labelslist_meniscus = evaluate_three('meniscus')
predslist_abnormal, labelslist_abnormal = evaluate_three('abnormal')

train axial
load_data meniscus axial
using alexnet ...
threshold is:  [1.9985895e+00 9.9858958e-01 9.4349897e-01 9.4247985e-01 9.1868448e-01
 9.1788006e-01 8.5264659e-01 8.5178310e-01 8.2650393e-01 8.2368714e-01
 7.9656619e-01 7.9593551e-01 7.9221290e-01 7.9073876e-01 7.5253862e-01
 7.5225055e-01 7.4259323e-01 7.4144578e-01 7.2857809e-01 7.2672981e-01
 7.0517379e-01 7.0158821e-01 7.0029652e-01 6.9964916e-01 6.9682825e-01
 6.9480932e-01 6.9259822e-01 6.9184762e-01 6.4329726e-01 6.4298970e-01
 6.3995057e-01 6.3987446e-01 6.3132024e-01 6.1770308e-01 6.0910326e-01
 6.0653198e-01 6.0490733e-01 5.9634608e-01 5.8880067e-01 5.8307576e-01
 5.7983649e-01 5.7755709e-01 5.7021338e-01 5.6861156e-01 5.4403299e-01
 5.3557831e-01 5.3258586e-01 5.1828957e-01 5.0888735e-01 4.9761319e-01
 4.9311790e-01 4.8925805e-01 4.8488188e-01 4.8224205e-01 4.7281483e-01
 4.6672267e-01 4.6429083e-01 4.6079263e-01 4.5783788e-01 4.4066018e-01
 4.3139261e-01 3.6412594e-01 3.5744852e-01 3.4717691e-01 3.4618962e-01
 3.4437

In [0]:
print('meniscus prediction values: train 3 oris, valid 3 oris, test 3 oris')
print(np.array(predslist_meniscus))
print(np.array(predslist_meniscus).shape)

meniscus prediction values: train 3 oris, valid 3 oris, test 3 oris
[[list([0.100345574, 0.9806981, 0.1235729, 0.8986333, 0.0355485, 0.18640155, 0.03114789, 0.068646275, 0.4062755, 0.2698479, 0.97818893, 0.005730499, 0.056822166, 0.69964916, 0.0023500065, 0.34520692, 0.67486537, 0.00095475675, 0.7525386, 0.99786985, 0.88478893, 0.021545384, 0.769134, 0.004418041, 0.13788033, 0.01839473, 0.0035090114, 0.72927105, 0.47281483, 0.4006957, 0.95046616, 0.09405548, 0.8193048, 0.961641, 0.48925805, 0.76937544, 0.7948358, 0.7753738, 0.68292975, 0.84379226, 0.17789549, 0.043149468, 0.80087435, 0.95334256, 0.042563546, 0.9596362, 0.013860825, 0.08101549, 0.022852363, 0.9573596, 0.093517765, 0.018515907, 0.9080531, 0.005763147, 0.9503528, 0.13167427, 0.017446375, 0.9615856, 0.105089754, 0.7578834, 0.016394233, 0.19601604, 0.09918726, 0.9085356, 0.0006561245, 0.121285304, 0.008411156, 0.8643106, 0.0019811767, 0.0748253, 0.95207834, 0.025742076, 0.8314501, 0.005516246, 0.028004363, 0.012359058, 0.00

In [0]:
#extracting the number from feature file names

test_string = 'name'
test_list = test_string.split()
xd = test_list[0].split('[')
xd2 = xd[2].split(']')
xd3 = float(xd2)

CLASSIFIERS

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc


def train_combo(epochs, ax_train, sag_train, cor_train, ground_truth_train, ax_valid, sag_valid, cor_valid, ground_truth_valid):
  #init
  valid_baseline_auc = 0
  rfc = RandomForestClassifier(n_estimators=1, criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1, 
                               min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, 
                               min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, 
                               warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)
  #updated_rfc = rfc
  
  X = np.concatenate((np.expand_dims(np.array(ax_train),axis=1), np.expand_dims(np.array(sag_train),axis=1), 
                      np.expand_dims(np.array(cor_train),axis=1)), axis=1)
  print('X shape', X.shape)
  y = np.array(ground_truth_train)
  print('Y shape', y.shape)
  auclist = []

  for epoch in range(epochs):
    old_rfc = rfc
    cv = StratifiedKFold(n_splits=5, random_state=123, shuffle=True)
    fprs, tprs, scores = [], [], []
    #fpr, tpr, threshold = metrics.roc_curve(labels, preds)
    #print("threshold is: ", threshold)
    #auc = metrics.auc(fpr, tpr)

    #for (train, test), i in zip(cv.split(X, y), range(5)):
    #    rfc.fit(X[train], y[train])
        #_, _, auc_score_train = compute_roc_auc(train)
        #fpr, tpr, auc_score = compute_roc_auc(test)
    #    y_preds = rfc.predict(X[test])
    #    fpr, tpr, threshold = metrics.roc_curve(y[test], y_preds)
    #    auc = metrics.auc(fpr, tpr)
    #    scores.append(auc)
    #    fprs.append(fpr)
    #    tprs.append(tpr)
    rfc.fit(X, y)

    #print('epoch ' + str(epoch), scores)
    X_valid = np.concatenate((np.expand_dims(np.array(ax_valid),axis=1), np.expand_dims(np.array(sag_valid),axis=1), 
                        np.expand_dims(np.array(cor_valid),axis=1)), axis=1)
    #print('X shape', X_valid.shape)
    y_valid = np.array(ground_truth_valid)
    #print('Y shape', y_valid.shape)
    y_valid_preds = rfc.predict(X_valid)
    fpr, tpr, threshold = metrics.roc_curve(y_valid, y_valid_preds)
    auc = metrics.auc(fpr, tpr)
    print('valid epoch ' + str(epoch), auc)

    if auc > valid_baseline_auc:
      valid_baseline_auc = auc
    else:
      rfc = old_rfc

    auclist.append(auc)

  return rfc, auclist

In [0]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc


def train_gridsearch(ax_train, sag_train, cor_train, ground_truth_train, ax_valid, sag_valid, cor_valid, ground_truth_valid):

  X = np.concatenate((np.expand_dims(np.array(ax_train),axis=1), np.expand_dims(np.array(sag_train),axis=1), 
                      np.expand_dims(np.array(cor_train),axis=1)), axis=1)
  y = np.array(ground_truth_train)

  n_estimators = [int(x) for x in np.linspace(start = 20, stop = 200, num = 10)]
  max_features = ['auto', 'sqrt']
  max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
  max_depth.append(None)
  min_samples_split = [2, 5, 10]
  min_samples_leaf = [1, 2, 4]
  bootstrap = [True, False]
  random_grid = {'n_estimators': n_estimators,
                'max_features': max_features,
                'max_depth': max_depth,
                'min_samples_split': min_samples_split,
                'min_samples_leaf': min_samples_leaf,
                'bootstrap': bootstrap}

  rfc = RandomForestClassifier()
  # Random search of parameters, using 3 fold cross validation, 
  # search across 100 different combinations, and use all available cores
  rfc_random = RandomizedSearchCV(estimator = rfc, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
  # Fit the random search model
  rfc_random.fit(X, y)

  X_valid = np.concatenate((np.expand_dims(np.array(ax_valid),axis=1), np.expand_dims(np.array(sag_valid),axis=1), 
                        np.expand_dims(np.array(cor_valid),axis=1)), axis=1)
  #print('X shape', X_valid.shape)
  y_valid = np.array(ground_truth_valid)
  #print('Y shape', y_valid.shape)
  y_valid_preds = rfc_random.predict(X_valid)
  fpr, tpr, threshold = metrics.roc_curve(y_valid, y_valid_preds)
  auc = metrics.auc(fpr, tpr)
  print('valid auc ' , auc)

  return rfc_random

In [0]:
# ACL
ACL_train_axial_preds = predslist_ACL[0][0]
ACL_train_sagittal_preds = predslist_ACL[1][0]
ACL_train_coronal_preds = predslist_ACL[2][0]

ACL_valid_axial_preds = predslist_ACL[3][0]
ACL_valid_sagittal_preds = predslist_ACL[4][0]
ACL_valid_coronal_preds = predslist_ACL[5][0]

#print(ACL_train_axial_preds)
#print(len(ACL_train_axial_preds), len(ACL_train_sagittal_preds), len(ACL_train_coronal_preds), len(train_ACL_labels))

rfc_random = train_gridsearch(ACL_train_axial_preds, ACL_train_sagittal_preds, ACL_train_coronal_preds, train_ACL_labels, 
                      ACL_valid_axial_preds, ACL_valid_sagittal_preds, ACL_valid_coronal_preds, valid_ACL_labels)


Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   10.1s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:   18.2s finished


valid auc  0.8106060606060606


In [0]:
from sklearn.neural_network import MLPClassifier

def train_nn(ax_train, sag_train, cor_train, ground_truth_train, ax_valid, sag_valid, cor_valid, ground_truth_valid):
  X_train = np.concatenate((np.array(ax_train), np.array(sag_train), np.array(cor_train)), axis=1)
  Y_train = np.array(ground_truth_train)
  X_valid = np.concatenate((np.array(ax_valid), np.array(sag_valid), np.array(cor_valid)), axis=1)
  Y_valid = np.array(ground_truth_valid)

  clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
  clf.fit(X_train, y_train)

  print("score", clf.score)
  y_pred = clf.predict(X_valid)
  fpr, tpr, threshold = metrics.roc_curve(Y_valid, y_pred)
  auc = metrics.auc(fpr, tpr)
  print('auc', auc)
  return X_train, Y_train

In [0]:
ACL_train_axial_preds = predslist_ACL[0][0]
ACL_train_sagittal_preds = predslist_ACL[1][0]
ACL_train_coronal_preds = predslist_ACL[2][0]

ACL_valid_axial_preds = predslist_ACL[3][0]
ACL_valid_sagittal_preds = predslist_ACL[4][0]
ACL_valid_coronal_preds = predslist_ACL[5][0]

x_train, y_train = train_nn(ACL_train_axial_preds, ACL_train_sagittal_preds, ACL_train_coronal_preds, train_ACL_labels, 
                      ACL_valid_axial_preds, ACL_valid_sagittal_preds, ACL_valid_coronal_preds, valid_ACL_labels)

AxisError: ignored

In [0]:
# neural net try 2

# Load libraries
import numpy as np
from keras import models
from keras import layers
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification

# Set random seed
np.random.seed(0)


# Create function returning a compiled network
def create_network():
    
    # Start neural network
    network = models.Sequential()

    # Add fully connected layer with a ReLU activation function
    network.add(layers.Dense(units=16, activation='relu', input_shape=(number_of_features,)))

    # Add fully connected layer with a ReLU activation function
    network.add(layers.Dense(units=16, activation='relu'))

    # Add fully connected layer with a sigmoid activation function
    network.add(layers.Dense(units=1, activation='sigmoid'))

    # Compile neural network
    network.compile(loss='binary_crossentropy', # Cross-entropy
                    optimizer='rmsprop', # Root Mean Square Propagation
                    metrics=['accuracy']) # Accuracy performance metric
    
    # Return compiled network
    return network

# Wrap Keras model so it can be used by scikit-learn
neural_network = KerasClassifier(build_fn=create_network, 
                                 epochs=10, 
                                 batch_size=100, 
                                 verbose=0)

features = x_train
target = y_train

cross_val_score(neural_network, features, target, cv=3)

In [0]:
def linechart(x, y, title, xlabel, ylabel):
  plt.figure()
  plt.title(title)
  plt.plot(x, y)
  plt.xlabel(xlabel)
  plt.ylabel(ylabel)
  plt.show()
  return

In [0]:
%matplotlib inline
linechart(np.arange(1000), np.array(auclist), 'title', 'epoch', 'auc')

In [0]:
def test_rfc(rfc, ax_pred, sag_pred, cor_pred, ground_truth):
  rfc_test = rfc
  X = np.concatenate((np.expand_dims(np.array(ax_pred),axis=1), np.expand_dims(np.array(sag_pred),axis=1), 
                      np.expand_dims(np.array(cor_pred),axis=1)), axis=1)
  print('X shape', X.shape)
  y = np.array(ground_truth)
  print('Y shape', y.shape)
  y_preds = rfc.predict(X)
  fpr, tpr, threshold = metrics.roc_curve(y, y_preds)
  auc = metrics.auc(fpr, tpr)
  return fpr, tpr, auc

In [0]:
a, b, c = test_rfc(acl_rfc, predslist_ACL[3][0], predslist_ACL[4][0], predslist_ACL[5][0], valid_ACL_labels)

print(a, b, c)

In [0]:
def test_meme(threshold, ax_pred, sag_pred, cor_pred, ground_truth):
  X = np.concatenate((np.expand_dims(np.array(ax_pred),axis=1), np.expand_dims(np.array(sag_pred),axis=1), 
                      np.expand_dims(np.array(cor_pred),axis=1)), axis=1)
  print('X shape', X.shape)
  y = np.array(ground_truth)
  print('Y shape', y.shape)
  newX = np.divide(np.sum(X, axis=1), 3)
  for index in range(newX.shape[0]):
    if round(newX[index]) > threshold:
      print('memes')
      newX[index] = 1
    else:
      newX[index] = 0    
  fpr, tpr, threshold = metrics.roc_curve(y, newX)
  auc = metrics.auc(fpr, tpr)
  return fpr, tpr, auc

In [0]:
#thresholds = np.divide(np.arange(10), 10)
thresholds = [1.0]
print(thresholds)

for threshold in thresholds:
  print(threshold)
  a, b, c = test_meme(threshold, predslist_ACL[3][0], predslist_ACL[4][0], predslist_ACL[5][0], valid_ACL_labels)
  print(a, b, c)

In [0]:
from sklearn.neural_network import MLPClassifier

def train_combo(diagnosis, ax_pred, sag_pred, cor_pred, ground_truth):
  X = np.concatenate((np.array(ax_pred), np.array(sag_pred), np.array(cor_pred)), axis=1)
  Y = 

clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
...                     hidden_layer_sizes=(5, 2), random_state=1)
...
>>> clf.fit(X, y)
MLPClassifier(alpha=1e-05, hidden_layer_sizes=(5, 2), random_state=1,
              solver='lbfgs')

In [0]:
#ax w/ aug
aug = True
epochs = 50
diagnosis = 'abnormal'
orientation = 'coronal'
lr = 1e-05
varray1, tarray1, testarray1 = train(rundir, diagnosis, orientation, epochs, lr, aug, gpu)
title = 'alexnet RAdam ' + diagnosis + ' ' + orientation + ' lr = ' + str(lr)
display_single(epochs, lr, varray1, tarray1, testarray1, title + ' aug = ' + str(aug) + " + new transforms", 'epoch', 'AUC', savedir)